In [1]:
%pip install tqdm


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoTokenizer, pipeline, PretrainedConfig
from optimum.onnxruntime import ORTModelForSeq2SeqLM
import onnxruntime

/home/ubuntu/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-04 14:19:49.224209: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-04 14:19:49.258337: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Load encoder model
encoder_session = onnxruntime.InferenceSession('quantized_afriteva_v2_large_unyo_dcyo_mix_onnx/encoder_model_quantized.onnx')

# Load decoder model
decoder_session = onnxruntime.InferenceSession('quantized_afriteva_v2_large_unyo_dcyo_mix_onnx/decoder_model_quantized.onnx')

In [4]:
model_id = "quantized_afriteva_v2_large_unyo_dcyo_mix_onnx"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [5]:
config = PretrainedConfig.from_json_file('quantized_afriteva_v2_large_unyo_dcyo_mix_onnx/config.json')

In [6]:
model = ORTModelForSeq2SeqLM(
    config=config,
    onnx_paths=['quantized_afriteva_v2_large_unyo_dcyo_mix_onnx/decoder_model_quantized.onnx','quantized_afriteva_v2_large_unyo_dcyo_mix_onnx/encoder_model_quantized.onnx'],
    encoder_session=encoder_session, 
    decoder_session=decoder_session, 
    model_save_dir='quantized_afriteva_v2_large_unyo_dcyo_mix_onnx',
    use_cache=False, 
)

In [7]:
onnx_translation = pipeline("translation_unyo_to_dcyo", model=model, tokenizer=tokenizer)

In [8]:
text_that_work_1 = "Bi a ba lo osuwon iseda, gbogbo gbolohun ede Yoruba ni a le pin si meji ni gbooro bayii"
text_that_work_2 = "Nje omo naa ni owo?"

In [8]:
# olorun oba o, ese o
# text = "Nje omo naa ni owo?" # mo lo si ile
# text = 'Ijoba orileede Naijiria ni ijoba China lo n koju aarun korona kii se pe won koriira tabi saida si omo Naijiria tabi alawodudu to n gbe Guangzhou ni China'
# text = 'Ijoba orileede Naijiria ni ijoba China lo n koju aarun korona'
# text = 'A ki i binu ori ka fi fila de ibadi.'
# text = 'mo fe jeun'
result = onnx_translation(text, max_length = 10000)
result

[{'translation_text': 'ǹkan'}]